<a href="https://colab.research.google.com/github/sushi15/Online-QA-System/blob/main/cqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [3]:
import itertools
import os 
import string 

import matplotlib.pylab as plt 
from matplotlib import pyplot
import numpy as np 
import pandas as pd 

import tensorflow as tf

import random 
import re
import nltk 
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

from nltk.corpus import stopwords
nltk.download('stopwords') 
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
lancaster_stemmer = LancasterStemmer()
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer() 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.metrics.pairwise import linear_kernel 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Load datasets

In [4]:
qa_pairs1 = pd.read_csv('cqa.csv', sep=',', header=0, encoding="utf-8") #Dataframe

In [5]:
qa_pairs1

,subject,question,answers
0,how to view old facebook statuses without clic...,"im pretty bored, so i wanted to view my old fa...",['facebook deletes old status updates after X ...
1,how can i make sure my facebook friends can se...,"on facebook, i went to privacy &gt; preview pr...","['check your privacy settings', ""If the photo ..."
2,how do i block someone on facebook without del...,"well, i just feel like telling you guys why :p...",['Hover over Settings at the top of the page a...
3,how to change settings on facebook?,if i comment or like on someones photo on face...,"[""I don't think you can"", 'if you go to your p..."
4,girls facebook everywhere and messy?,what are the girls talking about on facebook w...,"['?????? dont get your question! sorry', ""I go..."
...,...,...,...
1402,can i become a youtube partner without generat...,"i am part of an online community called pjtv, ...",['I\'ve been a partner for almost two years. G...
1403,which username is cooler? YouTube,[1] telbi\n\n[2] saulja\n,"['Telbi???? =/', 'Either one. Why not both? On..."
1404,when i send a video to me from u tube i get a ...,i open mail from u tube yesterday and don't kn...,['Error 404 usually means a file can\'t be fou...
1405,why won't youtube let me sign in!?,i was signed in to youtube but it randomly sig...,"['Try a different computer???', 'There could b..."


In [6]:
qa_pairs_l1 = qa_pairs1.values.tolist()

In [17]:
qa_pairs2 = pd.read_csv('test.csv', sep=',', header=0, encoding="utf-8") #Dataframe

In [18]:
qa_pairs2["subject"]

0                             What makes friendship click?
1                            Why does Zebras have stripes?
2                 What did the itsy bitsy sipder climb up?
3        What is the difference between a Bachelors and...
4                                    Why do women get PMS?
                               ...                        
59995    if you could be any fantasy figure, who would ...
59996    Tell me something about life most people don't...
59997                  Why are men always thinking of sex?
59998    est ce que DOMENECH est un entraineur: 1: de f...
59999                               No sound or low sound?
Name: subject, Length: 60000, dtype: object

In [19]:
del qa_pairs2["class"]

In [20]:
qa_pairs2

,subject,question,answers
0,What makes friendship click?,How does the spark keep going?,good communication is what does it. Can you m...
1,Why does Zebras have stripes?,What is the purpose or those stripes? Who do t...,this provides camouflage - predator vision is ...
2,What did the itsy bitsy sipder climb up?,NaN,waterspout
3,What is the difference between a Bachelors and...,NaN,One difference between a Bachelors and a Maste...
4,Why do women get PMS?,NaN,Premenstrual syndrome (PMS) is a group of symp...
...,...,...,...
59995,"if you could be any fantasy figure, who would ...",NaN,"The invisible man, I'd be straight into the gi..."
59996,Tell me something about life most people don't...,"Do you know anything about life, or words of w...",That there is a hell and everyone thinks their...
59997,Why are men always thinking of sex?,NaN,It's wired in our brain
59998,est ce que DOMENECH est un entraineur: 1: de f...,NaN,de foot mais pas pour être sélectionneur d'une...


In [21]:
qa_pairs_l2 = qa_pairs2.values.tolist()

In [22]:
qa_pairs_ext = [] 
for i in range(10): 
  qa_pairs_ext += qa_pairs_l2[i * 6000:i * 6000 + 500] 

In [23]:
len(qa_pairs_ext)

5000

## Models

In [ ]:
# question = input("Enter question: ") 

In [56]:
def getKeywords(question): 
    tagged = nltk.pos_tag(nltk.word_tokenize(question)) 
    # print(question)
    # print(tagged) 

    # The NLTK POS Tagger follows the Penn Treebank Project tag conventions 
    # Only the following kinds of words are extracted from the query as keywords 
    limit = ['FW', 'JJ', 'JJS', 'JJR', 'NN', 'NNS', 'NNP', 'NNPS', 'SYM', 'VB'] 
    keywords = [i[0] for i in tagged if i[1] in limit]
    # print(keywords) 
    return keywords 

In [ ]:
# keywords = getKeywords(question)

In [27]:

# ql = len(question)

In [28]:
def cosSim(question, qa_pairs_l): 
  res = {}
  subs =[]
  for i in range(len(qa_pairs_l)): 
    subs.append(qa_pairs_l[i][0])
  subs.insert(0, question) 
  tfidf = TfidfVectorizer().fit_transform(subs) 
  cosSims = linear_kernel(tfidf[0:1], tfidf).flatten() 
  i = 1 
  while i < len(subs): 
    subs[i] = i - 1 
    i += 1 
  subs = dict(zip(subs, cosSims)) 
  subs = dict(sorted(subs.items(), key = lambda item: item[1], reverse = True)) 
  # print(subs)
  subsk = list(subs.keys()) 
  subsv = list(subs.values()) 
  i = 1 
  while i < 10: 
    res[subsk[i]] = subsv[i] 
    i += 1 
  res = sorted(res.items(), key = lambda i: i[1], reverse = True) 
  return res 


In [29]:
def diceMetric(keywords, qa_pairs_l, ql): 
  res = {}
  for i in range(len(qa_pairs_l)): 
    count = 0 
    for j in keywords: 
      if j in qa_pairs_l[i][0]: 
        count += 1 
    count *= 2 
    count /= (ql + len(qa_pairs_l[i][0])) 
    res[i] = count 
  res = sorted(res.items(), key = lambda i: i[1]) 
  res = res[::-1]
  return res[:10]

In [30]:
# c = cosSim(question, qa_pairs_l2)

In [31]:
# print(c)

In [32]:
# d = diceMetric(keywords, qa_pairs_l2, ql)

In [33]:
# print(d)

In [34]:
stop_words = set(stopwords.words('english'))
if "the" in stop_words: 
  print("Yes")

Yes


In [35]:
def levenshtein(question, res, qa_pairs_l): 
  qc = question.translate(str.maketrans('', '', string.punctuation)) 
  tq = qc.split(" ")
  fin = {}
  for i in res: 
    curr = qa_pairs_l[i[0]][0].translate(str.maketrans('', '', string.punctuation)) 
    tcurr = curr.split(" ") 
    count = 0 
    for j in tq: 
      if j not in stop_words and j in tcurr: 
        count += 1 
    fin[i[0]] = count * i[1] 
    # fin[i[0]] = count 
  fin = sorted(fin.items(), key = lambda i: i[1]) 
  # print(fin)
  return fin[-1][0] 
  # return fin[0][0]

In [36]:
def levenshtein_mod(question, res, qa_pairs_l): 
  qc = question.translate(str.maketrans('', '', string.punctuation)) 
  tq = qc.split(" ")
  fin = {}
  for i in range(len(res)): 
    curr = qa_pairs_l[i][0].translate(str.maketrans('', '', string.punctuation)) 
    tcurr = curr.split(" ") 
    count = 0 
    for j in tq: 
      if j not in stop_words and j in tcurr: 
        count += 1 
    fin[i] = count
  fin = sorted(fin.items(), key = lambda i: i[1]) 
  # print(fin)
  return fin[-1][0] 
  # return fin[0][0] 

In [37]:
# ind = levenshtein(question, c, qa_pairs_l2) 

In [38]:
# ind = levenshtein(question, d, qa_pairs_l2) 

In [39]:
# print(ind)

In [40]:
# print(qa_pairs_l2[ind])

In [48]:
def encaps(question, qa_pairs_l): 
  keywords = getKeywords(question) 
  ql = len(question) 
  
  # Only Levenstein with count
  # ind = levenshtein_mod(question, qa_pairs_l, qa_pairs_l) 

  # Dice + Levenshtein without mult 
  # d = diceMetric(keywords, qa_pairs_l, ql) 
  # stop_words = set(stopwords.words('english')) 
  # ind = levenshtein(question, d, qa_pairs_l) # Reverse fin lines

  # Dice + Levenshtein with mult 
  # d = diceMetric(keywords, qa_pairs_l, ql) 
  # stop_words = set(stopwords.words('english')) 
  # ind = levenshtein(question, d, qa_pairs_l) 

  # CosSim only 
  # ind = cosSim(question, qa_pairs_l) 

  # CosSim + Levenshtein without mult 
  # c = cosSim(question, qa_pairs_l) 
  # ind = levenshtein(question, c, qa_pairs_l) # Reverse fin lines 

  # CosSim + Levenshtein with mult 
  c = cosSim(question, qa_pairs_l) 
  ind = levenshtein(question, c, qa_pairs_l) 
  
  # print(qa_pairs_l[ind])
  return ind 

In [ ]:
# ans = encaps(question, qa_pairs_l2)

In [ ]:
# print(ans)

In [ ]:
# print(qa_pairs_l2[ans][2])

## System

In [55]:
#@title Enter a question! { run: "auto", vertical-output: true }
Question = "What makes frienship Click?" #@param {type:"string"} 
Dataset = "Yahoo! Answers Topic Classification Dataset - 2007" #@param ["Yahoo! Answers Dataset for cQA - 2010", "Yahoo! Answers Topic Classification Dataset - 2007"]

qa_pairs_l = qa_pairs_l1 
if Dataset == "Yahoo! Answers Topic Classification Dataset - 2007": 
  qa_pairs_l = qa_pairs_l2 

ans = encaps(question, qa_pairs_l) 
print("The answer retrieved is: ") 
print(qa_pairs_l[ans][2])

What makes friendship Click
[('What', 'WP'), ('makes', 'VBZ'), ('friendship', 'NN'), ('Click', 'NNP')]
['friendship', 'Click']
The answer retrieved is: 
good communication is what does it.  Can you move beyond small talk and say what's really on your mind.  If you start doing this, my expereince is that potentially good friends will respond or shun you.  Then you know who the really good friends are.


## Evaluation

In [ ]:
def eval(qa_pairs_l): 
  count = 0 
  for i in range(len(qa_pairs_l)): 
    q = qa_pairs_l[i][0]
    ans = encaps(q, qa_pairs_l) 
    if qa_pairs_l[ans][0] == q: 
      count += 1 
  ratio = count / len(qa_pairs_l) 
  return ratio

In [ ]:
def eval_mod(qa_pairs_l): 
  precision = 0 
  recall = 0 
  for i in range(len(qa_pairs_l)): 
    q = qa_pairs_l[i][0]
    ans = encaps(q, qa_pairs_l) 
    if qa_pairs_l[ans][0] == q: 
      precision += 1 
      recall += 1 
    else: 
      shared = 0 
      qset = list(set(q.split(" "))) 
      ansset = list(set(qa_pairs_l[ans][0].split(" ")))
      for j in qset: 
        if j in ansset: 
          shared += 1 
      # print(shared / len(qa_pairs_l[ans][0]), i)
      precision += shared / len(qa_pairs_l[ans][0]) 
      recall += shared / len(q) 
  precision /= len(qa_pairs_l) 
  recall /= len(qa_pairs_l) 
  print(precision, recall) 
  fscore = 2 * (precision * recall) / (precision + recall) 
  return fscore 

In [ ]:
eval(qa_pairs_ext)

0.9986

In [ ]:
eval_mod(qa_pairs_ext)

0.9987204440827714 0.9987769863469861


0.9987487144146204